In [1]:
%load_ext autoreload

In [169]:
%autoreload
import torch
import matplotlib.pyplot as plt
from gpt.data import MNISTDataModule, CIFAR10DataModule
from collections import Counter
from torch.utils.data import Subset, DataLoader
import pickle

In [165]:
mnist = CIFAR10DataModule(batch_size=32)

In [129]:
import numpy as np

In [137]:
Counter(np.array(mnist.trainset.targets)[np.array(list(subset_idx))])

Counter({6: 3, 9: 3, 4: 3, 1: 3, 2: 3, 7: 3, 8: 3, 3: 3, 5: 3, 0: 3})

In [138]:
def setup_nshot(dm, n):
    subset_ctr = Counter({k: 3 for k in mnist.classes})
    subset_idx = set()
    
    ix = 0
    while subset_ctr.total() > 0:
        c = mnist.classes[mnist.trainset.targets[ix]]
        if subset_ctr[c] > 0:
            subset_ctr[c] -= 1
            subset_idx.add(ix)
        ix += 1

    return list(subset_idx)

In [140]:
Subset(mnist.trainset, setup_nshot(mnist, 2))

In [96]:
subset[mnist.classes[0]] += 1

In [93]:
seen_classes

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

In [60]:
tl = mnist.train_dataloader()

In [66]:
ys = []
for x, y in tl:
    ys.extend(y)
ys = map(lambda x: x.item(), ys)

In [78]:
tl.dataset.dataset.targets

tensor([5, 0, 4,  ..., 5, 6, 8])

In [62]:
torch.unique(torch.stack(ys))

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [170]:
with open('/scratch/gpfs/js5013/data/desi/batches/batch_bright_102400-104448.pkl', 'rb') as f:
    data = pickle.load(f)

In [171]:
data

[array([[ 3.807316  , -1.6357474 , -2.9151888 , ...,  4.9664726 ,
          5.4838347 ,  4.4018655 ],
        [-4.1529794 , 10.857753  , 12.699032  , ...,  1.6575221 ,
          1.3380166 ,  0.81630445],
        [ 3.8241894 ,  8.8644705 , 12.688596  , ...,  1.5328159 ,
          1.6082997 ,  1.4427842 ],
        ...,
        [15.7354145 , -0.35007486,  7.434158  , ...,  2.7656918 ,
          3.2473943 ,  2.1283998 ],
        [-5.2291813 , -7.6036277 , -9.241072  , ...,  0.36217168,
         -1.0106484 , -0.36796662],
        [46.456867  , 27.321129  , 21.284918  , ..., 13.632205  ,
         13.542756  , 14.266201  ]], dtype=float32),
 array([[2.56778132e-02, 1.54531430e-02, 1.23821869e-02, ...,
         3.35214233e+00, 4.58418655e+00, 6.74550581e+00],
        [1.96600761e-02, 1.06136594e-02, 6.60039764e-03, ...,
         1.93480647e+00, 2.43831444e+00, 4.51798487e+00],
        [4.05302271e-02, 2.64908783e-02, 1.88435633e-02, ...,
         7.12154579e+00, 7.47837543e+00, 1.08167877e+01]